# Neural Network

Nel campo dell'**Intelligienza Artificiale** (_AI_) le **Neural Network** (_NN_) rappresentano
uno degli approcci principali per permettere di compiere a sistemi automatici dei
task molto complessi e articolati che algoritmi classici (sequenziali) non sarebbero in grado di portare a termine se non con un grado di complessità elevato.

Le basi di questo nuovo approccio risalgono alla metà del XIX secolo quando per la prima volta furono proposti algoritmi per l'apprendimento automatico. Obiettivo di questo nuovo tipo di approccio era quello di creare _strutture_ in grado di modellare un determinato fenomeno e riproporne il comportamento sotto determinate condizioni.

Unità base di questa nuova struttura è il _**neurone**_. Con questo termine identifichiamo un oggetto in grado di simulare il comportamento di un neurone classico e di interconnettersi con altri per creare una rete. La gestione di queste uscite e di queste interconnessioni è stabilita dall'algoritmo di training che ha guidato la rete nella fase di addestramento appunto.

È curioso notare come la prospettiva iniziale di questo approccio era quella di creare macchine intelligienti, in grado di auto apprendere e riuscire ad eseguire in maniera intelligente i task che vengono sottomessi.

Un tipico esempio di struttura base di queste reti è il **percettrone**
![Percettrone](./img/percetrone.png)

Ogni singolo ingresso rappresenta il contributo che viene fornito da una particolare caratteristica del fenomeno atto dell'analisi. Tutti questi contributi apportano, in maniera pesata, il proprio apporto verso la particolare funzione di trasferimento del sistema, la cui uscita, in combinazione con una determinata funzione di attivazione, viene riportata al di fuori del sistema e rappresenta il valore interpretato dal neurone in base agli ingressi forniti.

Ponenedo in cascata e combinando tra loro più percettroni (o più genericamente più neuroni) generiamo quella che definiamo rete neurale
![ANN](./img/ann.png)

Nel corso degli anni l'interesse rivolto a questa nuova metodologia è andato via via scemando. Di pari passo però altri hanno proseguito lo studio delle reti neurali dandovi un taglio più matematico e sistemico grazie all'avvento di nuove metodologie come il **machine learinig**.


Grazie a questo nuovo tipo di approccio, i modelli attuali di reti neurali risultano molto più complessi e performanti dei loro antenati. A partire dalla metà degli anni 80 del secolo scorso, il campo dell'intelligineza arificiale, avendo acquisito maggiore autorevolezza e rigore scientifico, è diventato uno dei più floridi terreni di ricerca. I risultati e i campi applicativi acquisiscono, di giorno in giorno, sempre maggiore interesse. Siamo così passati ad analizzare, attraverso le reti neurali, problemi sempre più complessi: è l'avvento del **deep learning**.

Nelle reti neurali classiche moderne è possible riscontrare la presenza di qualche strato nascoto. Questi strati, denominati _hidden layers_, possono essere interpretati come il _cuore_ della rete stessa pocihé sono quelli che si occupano di interpretare le features sottomesse alla rete.

![NN](./img/simple_neural_network_header.jpg)

Con il deep learning, invece, siamo rapportati a problemi più complessi: _dal riconoscimento ed interpretazione del linguaggio naturale fino alla visione artificiale_. Nei modelli deep ogni singolo starto nascosto potrebbe essere paragonato ad una piccola rete neurale classica: ponenondone in cascata una all'altra possiamo ottnere modelli complessi per la gestione di task anche molto avanzati come ad esempio gli algoritmi di visione per l'_automotive_

![deep](./img/deep.png)